In [19]:
import torch

from sent_order.conll import Corpus, Document

from torch.nn import functional as F
from itertools import islice
from tqdm import tqdm_notebook

In [2]:
model = torch.load(
    '../../data/coref.4.bin',
    map_location={'cpu': 'cpu'},
)

In [3]:
model

Coref(
  (encode_doc): DocEncoder(
    (embeddings): WordEmbedding(39414, 300)
    (lstm): LSTM(300, 200, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.5)
  )
  (score_spans): SpanScorer(
    (attention): Scorer(
      (score): Sequential(
        (0): Linear(in_features=400, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
    (width_embeddings): DistanceEmbedding(9, 20)
    (sm): Scorer(
      (score): Sequential(
        (0): Linear(in_features=1120, out_features=150, bias=True)
        (1): ReLU()
        (2): Linear(in_features=150, out_features=150, bias=True)
        (3): ReLU()
        (4): Linear(in_features=150, out_features=1, bias=True)
      )
    )
  )
  (score_pairs): PairScorer(
    (dist_embeddings): DistanceEmbedding(9, 20)
    (sa): Scorer(
      (score): Sequential(
        (0): Linear(

In [10]:
c = Corpus.from_combined_file('../../data/test.conll')

In [24]:
def score(spans):
    return sum([F.softmax(span.sij, 0)[-1] for span in spans])

In [34]:
correct, total = 0, 0
for tokens1, tokens2 in tqdm_notebook(islice(c.sent_pair_tokens(), 200)):
    
    d1 = Document(tokens1+tokens2)
    d2 = Document(tokens2+tokens1)
    
    spans1 = model(d1)
    spans2 = model(d2)
    
    print(F.softmax(spans1[-1].sij).max())
    
    try:
        
        if score(spans1) > score(spans2):
            correct += 1

        total += 1
        
    except:
        pass

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/Users/dclure/Projects/plot-ordering/env/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


tensor(0.9890)
tensor(0.9997)
tensor(0.9997)
tensor(0.9981)
tensor(0.9996)
tensor(0.9865)
tensor(0.9683)
tensor(1.0000)
tensor(0.9783)
tensor(0.9900)
tensor(0.9994)
tensor(0.9998)
tensor(0.9986)
tensor(0.9999)
tensor(0.9997)
tensor(0.7247)
tensor(0.9995)
tensor(1.0000)
tensor(1.0000)
tensor(1.0000)
tensor(0.9983)
tensor(0.9928)
tensor(0.8857)
tensor(0.9809)
tensor(1.)
tensor(1.0000)
tensor(0.9997)
tensor(0.8017)
tensor(0.5379)
tensor(0.9309)
tensor(0.5170)
tensor(0.9997)
tensor(1.0000)
tensor(0.9929)
tensor(0.4915)
tensor(0.9998)
tensor(0.9999)
tensor(0.9997)
tensor(0.9742)
tensor(0.9759)
tensor(0.9998)
tensor(0.9984)
tensor(0.9985)
tensor(0.8407)
tensor(0.9999)
tensor(0.9971)
tensor(0.9981)
tensor(0.5289)
tensor(1.0000)
tensor(0.7148)
tensor(0.9805)
tensor(0.9984)
tensor(1.0000)
tensor(1.0000)
tensor(0.9936)
tensor(0.9976)
tensor(0.9990)
tensor(0.9995)
tensor(0.9999)
tensor(0.9992)
tensor(1.0000)
tensor(0.9995)
tensor(1.0000)
tensor(0.9999)
tensor(0.9993)
tensor(0.9938)
tensor(0.9998)

In [26]:
correct / total

0.505